## Nonlinear biomechanical model
A biomechanical model simulating the mouse ankle joint with a nonlinear muscle model. Model parameters of the skeletal segment are estimated based on the mouse forelimb. Joint excursion is constrained by exponential elastic stops. Small viscosity is added to improve the stability of the system, which is necessary for the elastic stops to work appropriately. Nonlinear properties of muscle are introduced to prevent the system from drifting when there is an offset between the flexor and extensor activities. 

### Import Libraries

In [1]:
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
device = 'cpu'

### Define muscle model functions 
* FL: force-legnth relationship
* FV: force-velocity relationship
* Fpe: passive elastic force

In [2]:
def FL_function(L):
    beta = 1.55
    omega = 0.75
    rho = 2.12
    
    FL = np.exp(-np.power(abs((np.power(L,beta) - 1)/(omega)),rho));
    return FL

In [3]:
def FV_con_function(L,V):
    Vmax = -9.15*2;
    cv0 = -5.7;
    cv1 = 9.18;
    
    FVcon = (Vmax - V)/(Vmax + (cv0 + cv1*L)*V);
    return FVcon

In [4]:
def FV_ecc_function(L,V):
    av0 = -1.4;
    av1 = 0.0;
    av2 = 0.0;
    bv = 0.72;
    FVecc = (bv - (av0 + av1*L + av2*np.power(L,2))*V)/(bv+V);
    
    return FVecc

In [5]:
def Fpe_function(L,V):
    c1_pe1 = 23.0*6
    k1_pe1 = 0.046
    Lr1_pe1 = 1.4
    eta = 0.0001;
    
    Fpe1 = c1_pe1 * k1_pe1 * np.log(np.exp((L - Lr1_pe1)/(k1_pe1))+1) + eta*V;
    
    return Fpe1

## Define simulation parameters 

In [6]:
Fs = 1000
h = 1/Fs
duration = 1

### Define inputs 
* u_1: input to dorsilexors (i.e. TA, EDL, EHL) 
* u_2: input to plantarflexors (i.e., MG, LG, SO, PL, FDL, TP)

In [7]:
# Set the amplitude of step inputs
u_1_amp = 0.9
u_2_amp = 1

# Generate step inputs to flexor and extensor
time_sim = torch.arange(0,duration,step = 1/Fs,dtype=torch.double).to(device)
u_1 = torch.zeros([int(duration*Fs),1],dtype=torch.double).to(device)
u_2 = torch.zeros([int(duration*Fs),1],dtype=torch.double).to(device)
u_1[int(0.1*Fs):] = u_1_amp
u_2[int(0.1*Fs):] = u_2_amp

### Define parameters of the biomechanical model

In [14]:
# Segment length
L = 0.015 
# Distance between the rotational axis and the center of the segment 
d = 0.015/2-0.005
# Segment mass
M = 0.00020154
# Segment inertia
I = 1/12.0*np.power(L,2)*M + np.power(d,2)*M
# Stiffness
K = 0.0
# Viscosity 
B = 0.00001

# Minimum joint angle
theta_min = 40*np.pi/180.0
# Maximum joint angle
theta_max = 110*np.pi/180.0

* Muscle 1: dorsilexors (i.e. TA, EDL, EHL) 
* Muscle 2: plantarflexors (i.e., MG, LG, SO, PL, FDL, TP)

Ref: Charles et al. (2011) [DOI: 10.1371/journal.pone.0147669]

In [25]:
# Maximum muscle force (N)
F0_1 = (7.64+1.52+0.35)*0.01*22.4
F0_2 = (7.43+14.57+1.42+3.67+7.3+2.15)*0.01*22.4
# 
Lce0_1 = 0.473*0.01; # Average fiber length of all dorsiflexors 
Lce0_2 = 0.389*0.01; # Average fiber length of all plantarflexors 
Lce_1 = Lce0_1/Lce0_1;
Lce_2 = Lce0_2/Lce0_2;
Vce_1 = 0;
Vce_2 = 0;
FLV_1 = FL_function(Lce_1)*FV_con_function(Lce_1,Vce_1);
FLV_2 = FL_function(Lce_2)*FV_con_function(Lce_2,Vce_2);
Fpe_1 = Fpe_function(Lce_1,Vce_1);
Fpe_2 = Fpe_function(Lce_2,Vce_2);

# Muscle time constant (ms)
tau_1 = 0.005;
tau_2 = 0.005;
# Moment arm (m)
r_m1 = 0.0006
r_m2 = -0.0015

## Find an equilibrium joint angle


## Find the maximum and minimum excursion of muscle length

In [19]:
dLm = -r_m1*(theta_max-90*np.pi/180.0)
Lce_min = (dLm + Lce0_1)/Lce0_1
Lce_min

0.8524034459201413

In [20]:
dLm = -r_m1*(theta_min-90*np.pi/180.0)
Lce_max = (dLm + Lce0_1)/Lce0_1
Lce_max

1.368991385199647

In [24]:
F0_2

8.18496